<a href="https://colab.research.google.com/github/cpcifuentes/NewProject/blob/master/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalación librerías

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from IPython.display import clear_output

!wget https://www.dropbox.com/s/54jfodpv1h4d0kl/pyxvis.zip
!unzip pyxvis.zip
!rm pyxvis.zip
print('PyXvis library downloaded.')
!pip install scipy==1.2
!pip3 install pybalu==0.2.5
!pip install ./pyxvis
# clear_output()
print('Librerías instaladas.')

--2022-07-04 00:41:19--  https://www.dropbox.com/s/54jfodpv1h4d0kl/pyxvis.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/54jfodpv1h4d0kl/pyxvis.zip [following]
--2022-07-04 00:41:20--  https://www.dropbox.com/s/raw/54jfodpv1h4d0kl/pyxvis.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc711006feeeedc15dde7cd53a00.dl.dropboxusercontent.com/cd/0/inline/BoZ1-Kp_K0g8zGbqUMFOczoGARxmDT3lMyDNpl7kd28pP086LleXMnm3gcIlEezZl9XLpIbcnE6_m2Po4LoU9Wu--UUcpj3PM7CZXm5EpFAaFbmg-dbWIL4k1LQEYuohVszdxYRIrWOqaGZbmx20PCRQA2kqswHodh5XRrVFk2CuPw/file# [following]
--2022-07-04 00:41:20--  https://uc711006feeeedc15dde7cd53a00.dl.dropboxusercontent.com/cd/0/inline/BoZ1-Kp_K0g8zGbqUMFOczoGARxmDT3lMyDNpl7kd28pP086LleXMnm3gcIlEezZl9XLpIbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./pyxvis
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for pyxvis: filename=pyxvis-0.0.1-py3-none-any.whl size=67637 sha256=f193d8f8042d89668c3944c6fb0a09676c05c11ea2ceea03778dcf5f2a0ceba9
  Stored in directory: /tmp/pip-ephem-wheel-cache-9s836eph/wheels/11/0e/e5/4e9331d67995eb55985def2c52ac94b3af5a21f262793a9b45
Successfully built pyxvis
Librerías instaladas.


In [ ]:
import numpy as np
import cv2 as cv
from   pyxvis.features.extraction import extract_features
from   sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from   sklearn.decomposition import FastICA
from   sklearn.cross_decomposition import PLSRegression
from   sklearn.neighbors import KNeighborsClassifier
from   sklearn.feature_selection import RFECV, RFE, SelectKBest, chi2, SelectFromModel 
from   sklearn.ensemble import RandomForestClassifier
from   sklearn.linear_model import LogisticRegression
from   sklearn.naive_bayes import GaussianNB
from   sklearn.neural_network import MLPClassifier
from   sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from   sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from   lightgbm import LGBMClassifier
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from   mlxtend.feature_selection import SequentialFeatureSelector as mlxsfs
from   pybalu.feature_transformation import normalize, pca
from   pybalu.feature_selection import clean, sfs, exsearch
from   pybalu.feature_extraction import lbp_features, haralick_features, gabor_features, hog_features, basic_int_features
from   tqdm.auto import tqdm
clear_output()
print('Librerías cargadas.')

NameError: ignored

# Sección nueva

## Data Getter
Given a name, it will download a thing

In [ ]:
import os

links = {
    "Bicicletas" :"https://www.dropbox.com/s/nznuspgkuk9pax3/Bicicletas.zip",
    "Cachipun" :"https://www.dropbox.com/s/cwr55d7mik3b9gt/Cachipun.zip",
    "Espinas" :"https://www.dropbox.com/s/xb3qhvukiquqk3k/Espinas.zip",
    "Letras" :"https://www.dropbox.com/s/uhpnih2ov6kn3l6/Letras.zip",
    "Lunares" :"https://www.dropbox.com/s/atml3zlyvt0b83z/Lunares.zip",
}

def get_db(name:str, links=links) -> None:
  '''
    Recibe un nombre y un diccionario de links, busca en el diccionario la direccion en la que esta
    la bases de datos, descarga el archivo zip, lo descomprime con unzip y elimina el comprimido

      name[str]: El nombre del archivo
      links[dict](Opcional): Diccionario con links
  '''
  link = links[name]
  os.system(f"wget {link}")
  os.system(f"unzip {name}.zip")
  os.system(f"rm -rf {name}.zip")

In [ ]:
get_db('Lunares')

In [ ]:

# Geometric feature Extractors
def get_binaryFeature(image:np.ndarray, feature:str) -> np.array:
  mean = np.mean(image)
  R = (image > mean)*1
  return extract_features(feature, bw=image)

def get_hu(image:np.ndarray) -> np.array:
  return cv.HuMoments(cv.moments(image)).flatten()

def get_flusser(image: np.ndarray) -> np.array:
  return get_binaryFeature(image, 'flusser')

def get_contrast(image: np.ndarray) -> np.array:
  return get_binaryFeature(image, 'contrast')

def get_fourierDes(image: np.ndarray) -> np.array:
  return get_binaryFeature(image, 'fourierdes')

def get_gupta(image: np.ndarray) -> np.array:
  return get_binaryFeature(image, 'gupta')

def get_basicGeo(image:np.ndarray) -> np.array:
  return get_binaryFeature(image, 'basicgeo')

def get_ellipse(image:np.ndarray) -> np.array:
  return get_binaryFeature(image, 'ellipse')

def get_centroid(image:np.ndarray) -> np.array:
  return get_binaryFeature(image, 'centroid')

def get_symmetry(image):
  radial=0
  h = 0
  v = 0
  for i in range(int(image.shape[1])):
    for j in range(int(image.shape[0])):
      i_alt = image.shape[1] - i - 1
      h += image[j,i] == image[j,i_alt]
      
  for i in range(int(image.shape[1])):
    for j in range(int(image.shape[0])):
      j_alt = image.shape[0] - j - 1
      v += image[j,i] == image[j_alt,i]

  for i in range(int(image.shape[1])):
    for j in range(int(image.shape[0])):
      i_alt = image.shape[1] - i - 1
      j_alt = image.shape[0] - j - 1
      radial += image[j,i] == image[j_alt,i_alt]

  h /= image.shape[0]*image.shape[1]
  v /= image.shape[0]*image.shape[1]
  radial /= image.shape[0]*image.shape[1]
  return np.array([v, h, radial])

def get_area(image:np.ndarray) -> np.array:
  mean = np.mean(image)
  return np.array([np.sum(image>mean)/(image.shape[0]*image.shape[1])])

## Texture feature extractors
def get_lbp(image:np.ndarray,hdiv,vdiv,mapping) -> np.array:
  return lbp_features(image, hdiv=hdiv, vdiv=vdiv, mapping=mapping)

def get_haralick(image:np.ndarray,dis) -> np.array:
  return haralick_features(image, distance=dis)

def get_gabor(image:np.ndarray,rot,dil) -> np.array:
  return gabor_features(image, rotations=rot, dilations=dil)

def get_hog(image:np.ndarray,hdiv,vdiv,bins) -> np.array:
  return hog_features(image, v_windows=vdiv, h_windows=hdiv, n_bins=bins)

def get_basicint(image:np.ndarray) -> np.array:
  return basic_int_features(image)

def get_gabor(image:np.ndarray, rot, dil) -> np.array:
  return gabor_features(image, rotations=rot, dilations=dil)

# Funciones encargadas de la obtención de imagenes
def open_image(path, channels='single') -> np.ndarray:
  if channels == 'single':
    return cv.imread(path, cv.IMREAD_GRAYSCALE)
  return cv.imread(path)

def get_image(kind, idx, channel) -> np.ndarray:
  ch = ['peaton', 'fondo', 'bicicleta'][kind-1]
  path = f'./Bicicletas/{kind:02}_{ch}/person_{kind:02}_{idx:04}.png'
  return open_image(path, channel)

In [ ]:
import json

class AutoPR:
  # Utilizando parametros en formato json
  def __init__(self, param_route: str) -> None:
    try:
      with open(param_route, 'r') as f:
        params = json.load(f)['Parameters']
        print(f"Loading Parameters from {param_route}")
    except Exception as error:
      print(f'While reading the parameters file, an exception ocurred:\n {error}')
    else:
      self.dataset = params.get('Dataset')
      print("\t- Dataset:",self.dataset)
      # Verificar si es None
      self.geo_features = params.get('GeoFeatures') 
      print("\t- Geometric Features:",self.geo_features)
      # Verificar si es None
      self.tex_features = params.get('TexFeatures')
      print("\t- Texture Features:",self.tex_features)
      # Verificar si es None
      self.feat_comb = params.get('FeatureCombination')
      print("\t- Feature Combination:",self.feat_comb)
      self.classifiers = params.get('Classifiers')
      print("\t- Classifiers:",self.classifiers)
      self.evaluation = params.get('Evaluation')
      print("\t- Evaluation:",self.evaluation)
      self.evaluation = self.evaluation[0][1][0]
      self.trained_cls = {}

      self.feature_extractor_functions = {
        'bas': get_basicint,
        'har': get_haralick,
        'hog': get_hog,
        'lbp': get_lbp,
        'con': get_contrast,
        'fou': get_fourierDes,
        'are': get_area,
        'cen': get_centroid,
        'ell': get_ellipse,
        'flu': get_flusser,
        'geo': get_basicGeo,
        'gup': get_gupta,
        'hu': get_hu,
        'sym': get_symmetry
      }

      # El propósito de esta variable es almacenar a las imagenes para cada llamada de feature no encontrada
      self.db = None

  def load_db(self,path):
    
  def extract_features(self, feature:list) -> np.array:
    # TODO
    args = feature[1]
    name = feature[0].lower()

    # Aqui hay que definir la base de datos a entregarle a las funciones, si no está hay que obtenerla
    # Leer imagen y almacenarla en self.db
    X = None

    # Obtenemos la funcion a ejecutar
    func = self.feature_extractor_functions[name]
  
    # Se retorna una matriz de caracteristicas resultado de la aplicacion de la funcion
    return func(X, *args)

  def load_features(self) -> np.ndarray:
    '''
    Carga las características especificadas por los atributos (geo_features,tex_features), desde las siguientes
    carpetas: features/Geo/Base_de_Datos/ y/o features/Tex/Base_de_Datos/.
    '''
    # TODO: calculate features if not saved
    features = []

    if self.geo_features:
      geo_features = []
      for feature in self.geo_features:
        path = f'./drive/Shareddrives/Patrones/features/{self.dataset}/X_{feature[0][:3].lower()}.npy'
        if os.path.isfile(path):
          geo_features.append(np.load(path))
        else: 
          # TODO
          X = self.extract_features(feature)
          np.save(X, path)
          geo_features.append(X)
      if geo_features:
        features.extend(geo_features)

    if self.tex_features:
      tex_features = []
      for feature in self.tex_features:
        # Se extraen el nombre y los argumentos y se arma el string
        name = feature[0][:3].lower()
        args = list(map(lambda x: str(x), feature[1]))
        path = f'./drive/Shareddrives/Patrones/features/{self.dataset}/X_{name}_{"_".join(args)}.npy'
        if os.path.isfile(path):
          tex_features.append(np.load(path))
        else: 
          # TODO
          X = self.extract_features(feature)
          np.save(X, path)
          tex_features.append(X)
      if tex_features:
        features.extend(tex_features)

    X = np.concatenate(features,axis=1)
    y = np.load(f'./drive/Shareddrives/Patrones/features/{self.dataset}/Y_{self.dataset}.npy')
    return X, y

  def feat_select_trans(self, X, y):
    '''
    Aplica las strategies de seleccion y transformacion indicadas
    '''
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.30, random_state=42,stratify=y)
    # clean
    sclean = clean(Xtrain)

    Xtrain_clean = Xtrain[:, sclean]
    Xtest_clean = Xtest[:, sclean]

    # normalizacion min max
    ma = Xtrain_clean.max(0)
    mi = Xtrain_clean.min(0)
    md = ma-mi
    a = 1/md
    b = -mi/md

    Xtrain_norm = Xtrain_clean * a + b
    Xtest_norm = Xtest_clean * a + b

    strategies = []
    print("Feature Combination")
    if self.feat_comb:
      # selection_transformation = lista de cada seleccion o transformacion a realizar
      for selection_transformation in self.feat_comb:
        print("\t- Combination",selection_transformation)
        
        Xtrain_copy = Xtrain_norm.copy()
        Xtest_copy = Xtest_norm.copy()
        name = ""

        # strat = nombre de la seleccion/transformacion y sus parametros
        for strat in selection_transformation:
          print("\t- Strategy",strat)
          if isinstance(strat[1],list):
            name = name + strat[0] + "_" + str(strat[1][0])
            print("\t\t- Params",strat[1][0])
          else:
            name = name + strat[0]
          if strat[0] == "SFS":
            p = strat[1][0]
            Xtrain_copy, Xtest_copy = self.sfs(Xtrain_copy, Xtest_copy, ytrain, p)
          
          elif strat[0] == "exsearch":
            p = strat[1][0]
            Xtrain_copy, Xtest_copy = self.exsearch(Xtrain_copy, Xtest_copy, ytrain, p)

          elif strat[0] == "RFECV":
            Xtrain_copy, Xtest_copy = self.rfecv(Xtrain_copy, Xtest_copy, ytrain)

          elif strat[0] == "RFE":
            Xtrain_copy, Xtest_copy = self.rfecv(Xtrain_copy, Xtest_copy, ytrain)

          elif strat[0] == "SBS":
            p = strat[1][0]
            Xtrain_copy, Xtest_copy = self.sbs(Xtrain_copy, Xtest_copy, ytrain, p)

          elif strat[0] == "selectkbest":
            Xtrain_copy, Xtest_copy = self.select_k_best(Xtrain_copy, Xtest_copy, ytrain)

          elif strat[0] == "PCA":
            p = strat[1][0]
            Xtrain_copy, Xtest_copy = self.pca(Xtrain_copy, Xtest_copy, p)

          elif strat[0] == "ICA":
            p = strat[1][0]
            Xtrain_copy, Xtest_copy = self.ica(Xtrain_copy, Xtest_copy, ytrain, n=p)

          elif strat[0] == "PLSR":
            p = strat[1][0]
            Xtrain_copy, Xtest_copy = self.plsr(Xtrain_copy, Xtest_copy, n=p)

        strategies.append((name,Xtrain_copy, Xtest_copy))
        
    return strategies, ytrain, ytest

  # Funciones seleccion de caracteristicas
  def sfs(self, Xtrain, Xtest, ytrain, p=10):
    sel = sfs(Xtrain, ytrain, p)
    Xtrain_sel = Xtrain[:,sel]
    Xtest_sel  = Xtest[:,sel]
    return Xtrain_sel, Xtest_sel

  def exsearch(self, Xtrain, Xtest, ytrain, p=10):
    sel = exsearch(Xtrain, ytrain, p)
    Xtrain_sel = Xtrain[:,sel]
    Xtest_sel  = Xtest[:,sel]
    return Xtrain_sel, Xtest_sel

  def rfecv(self, Xtrain, Xtest, ytrain, kern="linear", stp=1, v=5):
    estimator = SVC(kernel=kern)

    selector  = RFECV(estimator, step=stp,cv=v)
    selector  = selector.fit(Xtrain, ytrain)
    sel       = np.nonzero(selector.support_)[0]

    Xtrain_sel = Xtrain[:,sel]
    Xtest_sel  = Xtest[:,sel]
    return Xtrain_sel, Xtest_sel

  def rfe(self, Xtrain, Xtest, ytrain, kern="linear"):
    estimator = SVC(kernel=kern)

    selector  = RFE(estimator)
    selector  = selector.fit(Xtrain, ytrain)
    sel       = np.nonzero(selector.support_)[0]

    Xtrain_sel = Xtrain[:,sel]
    Xtest_sel  = Xtest[:,sel]
    return Xtrain_sel, Xtest_sel

  def sbs(self, Xtrain, Xtest, ytrain, estimator, p=10, v=10):
    forward = False
    sbs = mlxsfs(estimator, k_features=p,forward=forward,floating=False,scoring='accuracy',cv=v)

    sbs = sbs.fit(Xtrain, ytrain)
    sel = list(sbs.k_feature_idx_)

    Xtrain_sbs = Xtrain[:,sel]
    Xtest_sbs  = Xtest[:,sel]
    return Xtrain_sbs, Xtest_sbs

  def select_k_best(self, Xtrain, Xtest, ytrain, p=10):
    model =  SelectKBest(chi2,k=p)

    eps = 1e-8
    model.fit(Xtrain+eps, ytrain)  

    sel        = np.nonzero(model.get_support())[0]

    Xtrain_sel = Xtrain[:,sel]
    Xtest_sel  = Xtest[:,sel]
    return Xtrain_sel, Xtest_sel

  # transformacion de caracteristicas
  def pca(self, Xtrain, Xtest, n=10):
    Xtrain_pca, _, A, Xm, _ = pca(Xtrain, n_components=n)
    Xtest_pca = np.matmul(Xtest - Xm, A)
    return Xtrain_pca, Xtest_pca

  def ica(self, Xtrain, Xtest, ytrain, n=10, rs=0):
    ica = FastICA(n_components=n, random_state=rs)
    ica.fit(Xtrain, ytrain)    
    Xtrain_ica = ica.transform(Xtrain)
    Xtest_ica  = ica.transform(Xtest)
    return Xtrain_ica, Xtest_ica

  def plsr(self, Xtrain, Xtest, ytrain, n=10):
    plsr = PLSRegression(n_components=n)    
    plsr.fit(Xtrain, ytrain)    
    Xtrain_plsr = plsr.transform(Xtrain)
    Xtest_plsr  = plsr.transform(Xtest)
    return Xtrain_plsr, Xtest_plsr

  def model_selection(self):
    # Separar datos entre train y test
    n_folds = self.evaluation
    # proportion_crossval = .8
    cv_value = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=1)

    # Por cada selector/transformador de caracteristicas
    accuracy = {}

    # Load
    X, y = self.load_features()

    # feat_select_trans ya hace split entre train y test
    strategies, y_train, y_test = self.feat_select_trans(X, y)

    self.classification()

    for i in range(len(strategies)):
      # El selector de características, se asume que este no está calculado ya
      # En caso de que ya lo esté, reemplazar por Xprime = self.selectors[i][<train>]
      # Strategies contiene a las tuplas Xtrain y Xtest 
      strat_name, X_train, X_test = strategies[i]
      for name in self.trained_cls.keys():
        accuracy[i,name] = np.mean(cross_val_score(
            self.trained_cls[name], 
            X_train, 
            y_train, 
            scoring='accuracy', 
            cv = cv_value
        ))
    # Mejor indice de la estrategia y llave de clasificador
    iMax, nameMax = 0, ""
    accMax = -1
    for (strat_index,model_name),acc in accuracy.items():
      if accMax < acc:
        iMax, accMax, nameMax = strat_index, acc, model_name
    best_strat_name, X_train_prime, X_test_prime = strategies[iMax] # Es tupla Xtrain, Xtest
    acc = self.holdout(X_train_prime, y_train, X_test_prime, y_test, self.trained_cls[nameMax])
    # Retorna nombre de la mejor estrategia y clasificador, con sus respectivos parametros, el modelo y su accuracy
    return best_strat_name, nameMax, self.trained_cls[nameMax], acc

  def holdout(self, X_train, y_train, X_test, y_test, cls):
    '''
    Recibe X_train y X_test junto con y_test y obtiene el accuracy.
    ''' 
    cls.fit(X_train,y_train)
    y_pred = cls.predict(X_test)
    return sum([ 1 if y_pred[i] == y_test[i] else 0 for i in range(y_test.shape[0])])/y_test.shape[0]

  # Entrenamiento del clasificador
  def classification(self):

    for classifier in self.classifiers:
      name, hiperpar = classifier

      if name == 'SVM':
        model = SVC(kernel=hiperpar[0])
      elif name == 'KNN':
        model = KNeighborsClassifier(n_neighbors=hiperpar[0])
      elif name == 'RandomForest':
        model = RandomForestClassifier(max_depth=hiperpar[0], n_estimators=hiperpar[1])
      elif name == 'NaiveBayes':
        model = GaussianNB()
      elif name == 'NN':
        model = MLPClassifier(alpha=hiperpar[0], hidden_layer_sizes=hiperpar[1], 
                      random_state=1,max_iter=1000)
      elif name == 'LogisticRegression':
        model = LogisticRegression(C=hiperpar[0],solver="lbfgs",max_iter=1000)
      elif name == 'QDA':
        model = QDA()

      key = name + "_" + "_".join([str(par) for par in hiperpar])
      self.trained_cls[key] = model


In [ ]:
root = './drive/Shareddrives/Patrones/parametros_finales.json'

prueba = AutoPR(root)
X, y = prueba.load_features()
strat_name, class_name, mo, acc = prueba.model_selection()
print("Results:")
print("\t- Best Strategy:",strat_name)
print("\t- Best Classifier:",class_name)
print("\t- Accuracy:",acc)

Loading Parameters from ./drive/Shareddrives/Patrones/parametros_finales.json
	- Dataset: Lunares
	- Geometric Features: None
	- Texture Features: [['LBP', [3, 3, 'r']], ['LBP', [3, 3, 'g']], ['LBP', [3, 3, 'b']], ['Haralick', [2]], ['Haralick', [3]]]
	- Feature Combination: [[['SFS', [50]], ['PCA', [10]]], [['SFS', [10]], ['ICA', [5]]]]
	- Classifiers: [['SVM', ['linear']], ['SVM', ['rbf']], ['NN', [1e-05, [12, 12]]], ['NN', [1e-05, [24, 24]]], ['KNN', [15]], ['QDA', []]]
	- Evaluation: [['CV', [10]]]
./drive/Shareddrives/Patrones/features/Lunares/X_lbp_3_3_r.npy


ValueError: ignored